In [6]:
import json
from pathlib import Path
from typing import List, Optional, Dict, Any, Union
from peewee import *
import os

In [7]:
import importlib
import core
importlib.reload(core)
from core import db, Question
from core import parse_key, get_key, key_to_query_filter

In [5]:
from core import get_categories
get_categories('difficulty:easy')

difficulty ['easy']
filter_dict {'difficulty': ['E']}
module all
class all
skill all
difficulty ['easy']
active all
filter_dict {'difficulty': ['E']}


[{'module': 'english',
  'module_count': 324,
  'module_first_question_id': '3543e6e2',
  'class': 'CAS',
  'class_count': 107,
  'class_first_question_id': '84b5125b',
  'skill': 'CTC',
  'skill_count': 14,
  'skill_first_question_id': '02fd3da7'},
 {'module': 'english',
  'module_count': 324,
  'module_first_question_id': '3543e6e2',
  'class': 'CAS',
  'class_count': 107,
  'class_first_question_id': '84b5125b',
  'skill': 'TSP',
  'skill_count': 20,
  'skill_first_question_id': 'c966ad55'},
 {'module': 'english',
  'module_count': 324,
  'module_first_question_id': '3543e6e2',
  'class': 'CAS',
  'class_count': 107,
  'class_first_question_id': '84b5125b',
  'skill': 'WIC',
  'skill_count': 73,
  'skill_first_question_id': '84b5125b'},
 {'module': 'english',
  'module_count': 324,
  'module_first_question_id': '3543e6e2',
  'class': 'EOI',
  'class_count': 57,
  'class_first_question_id': '660d50dc',
  'skill': 'SYN',
  'skill_count': 19,
  'skill_first_question_id': '264e7415'},
 

In [2]:
# DB Connect
db = SqliteDatabase('data/index.db')
db.connect()

True

In [15]:
# db.close()
# os.remove('data/index.db')

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'data/index.db'

In [3]:
# Load data
data_path = Path('.').parent / 'data'
db_path = data_path / r"cb-digital-questions.json"
with db_path.open(encoding='utf-8') as fp:
    questions = json.load(fp)

with (data_path / r"lookup.json").open(encoding='utf-8') as fp:
    filter_data = json.load(fp)

    for k, v in [('mathLiveItems', 'math'), ('readingLiveItems', 'english')]:
        filter_data[v] = filter_data[k]
        del filter_data[k]

question_id_map = {}
for q_id, q in questions.items():
    question_id_map[q['questionId']] = q_id

In [31]:
# Create Tables
db.drop_tables([Question])
db.create_tables([Question])

In [32]:
# Add Question to DB
questions_model = []
for q_id, q_meta in questions.items():
    question = Question(
        q_id=q_id,
        questionId=q_meta['questionId'],
        skill_cd=q_meta['skill_cd'],
        score_band_range_cd=q_meta['score_band_range_cd'],
        uId=q_meta['uId'],
        program=q_meta['program'],
        external_id=q_meta['external_id'],
        primary_class_cd=q_meta['primary_class_cd'],
        difficulty=q_meta['difficulty'],
        module=q_meta['module'],
        active=q_meta['external_id'] in filter_data[q_meta['module']]
    )
    questions_model.append(question)

Question.bulk_create(questions_model)

In [6]:
Question.select().where(Question.active == False).count()

956

In [7]:
def questions_filter(where = None) -> List[Question]:
    print(Question.select().where(where).count())
    
    # query = Question.select().where(
    #     Question.module == module,
    #     Question.skill_cd == skill_cd,
    #     Question.primary_class_cd == primary_class_cd,
    #     Question.active.in_(active)
    # )
    # 
    # return list(query)

In [8]:
questions_filter(module='math')

1079


In [70]:


print(parse_key(''))

parse_key(get_key(module='math', primary_class_cd='CAS', skill_cd='all', difficulty='all', active='all'))

{}
{}
module math
class CAS
skill all
difficulty ['all']
active all
{'module': 'math', 'class': 'CAS'}


{'module': 'math', 'class': 'CAS'}

Module: english, Count: 938
Module: math, Count: 1079
Module: english, Class: CAS, Count: 247
Module: english, Class: EOI, Count: 189
Module: english, Class: INI, Count: 291
Module: english, Class: SEC, Count: 211
Module: math, Class: H, Count: 343
Module: math, Class: P, Count: 306
Module: math, Class: Q, Count: 277
Module: math, Class: S, Count: 153
Module: english, Class: CAS, Skill: CTC, Count: 45
Module: english, Class: CAS, Skill: TSP, Count: 72
Module: english, Class: CAS, Skill: WIC, Count: 130
Module: english, Class: EOI, Skill: SYN, Count: 103
Module: english, Class: EOI, Skill: TRA, Count: 86
Module: english, Class: INI, Skill: CID, Count: 66
Module: english, Class: INI, Skill: COE, Count: 151
Module: english, Class: INI, Skill: INF, Count: 74
Module: english, Class: SEC, Skill: BOU, Count: 107
Module: english, Class: SEC, Skill: FSS, Count: 104
Module: math, Class: H, Skill: H.A., Count: 65
Module: math, Class: H, Skill: H.B., Count: 87
Module: math, Class: H, Skill: H.C., 

In [15]:
# category -> subcategory
# primary_class_cd -> skill_cd

query = (
    Question.select(
        Question.primary_class_cd.alias("primary_class"),
        Question.skill_cd.alias("skill"),
        # fn.COUNT(Question.primary_class_cd).filter(Question.primary_class_cd == Question.primary_class_cd).alias("primary_class_count"),
        # SQL("(SELECT Count(*) FROM question WHERE primary_class_cd = primary_class_cd) as primary_class_count"),
        (Question.select(SQL('COUNT(*) AS primary_class_count')).where(Question.primary_class_cd == Question.primary_class_cd).order_by(Question.primary_class_cd).group_by(Question.primary_class_cd)).alias("primary_class_count"),
        
        # fn.COUNT(Question.primary_class_cd).alias("primary_class_count"),
        fn.COUNT(Question.skill_cd).alias("skill_count"),
    )
    # .where(Question.difficulty == 'E')
    .order_by(Question.module, Question.primary_class_cd, Question.skill_cd)
    .group_by(Question.module, Question.skill_cd)
)

# Execute the query and iterate through results
for row in query:
    print(
        f"primary_class: {row.primary_class}, skill: {row.skill}, primary_class_count: {row.primary_class_count}, skill_count: {row.skill_count}"
    )

primary_class: CAS, skill: CTC, primary_class_count: 247, skill_count: 45
primary_class: CAS, skill: TSP, primary_class_count: 247, skill_count: 72
primary_class: CAS, skill: WIC, primary_class_count: 247, skill_count: 130
primary_class: EOI, skill: SYN, primary_class_count: 247, skill_count: 103
primary_class: EOI, skill: TRA, primary_class_count: 247, skill_count: 86
primary_class: INI, skill: CID, primary_class_count: 247, skill_count: 66
primary_class: INI, skill: COE, primary_class_count: 247, skill_count: 151
primary_class: INI, skill: INF, primary_class_count: 247, skill_count: 74
primary_class: SEC, skill: BOU, primary_class_count: 247, skill_count: 107
primary_class: SEC, skill: FSS, primary_class_count: 247, skill_count: 104
primary_class: H, skill: H.A., primary_class_count: 247, skill_count: 65
primary_class: H, skill: H.B., primary_class_count: 247, skill_count: 87
primary_class: H, skill: H.C., primary_class_count: 247, skill_count: 78
primary_class: H, skill: H.D., prima

In [19]:
question_class_counts = (
    Question.select(
        Question.module, 
        Question.primary_class_cd, 
        fn.COUNT().alias('count'),
        Question.questionId.alias('first_question_id_class')
    )
    # .where(Question.difficulty == 'E')
    .order_by(Question.module, Question.primary_class_cd)
    .group_by(Question.primary_class_cd)
)

for question in question_class_counts:
    print(f"Module: {question.module}, Class: {question.primary_class_cd}, Count: {question.count}, QuestionId: {question.first_question_id_class}")

Module: english, Class: CAS, Count: 247, QuestionId: 84b5125b
Module: english, Class: EOI, Count: 189, QuestionId: afec1a70
Module: english, Class: INI, Count: 291, QuestionId: f1bfbed3
Module: english, Class: SEC, Count: 211, QuestionId: de55ec71
Module: math, Class: H, Count: 343, QuestionId: 002dba45
Module: math, Class: P, Count: 306, QuestionId: beca03de
Module: math, Class: Q, Count: 277, QuestionId: 85939da5
Module: math, Class: S, Count: 153, QuestionId: 6d99b141


In [71]:
def questions_id(filter_s: str = '') -> List[str]:
    query = Question.select(Question.questionId).where(
        *(key_to_query_filter(filter_s) or [None])
    )
    return [q.questionId for q in query]

def questions_count(filter_s: str = '') -> int:
    return Question.select(Question.questionId).where(
        *(key_to_query_filter(filter_s) or [None])
    ).count()

len(questions_id('module:math-class:all-skill:all-difficulty:all-active:bluebook'))

module math
class all
skill all
difficulty ['all']
active bluebook
{'module': 'math', 'active': True}


508

In [13]:
Question.select().where(Question.module=='english', Question.active==None).count()

0

In [14]:
def a(**kwargs):
    print(kwargs)
    
a(class=1)

SyntaxError: invalid syntax (2070734320.py, line 4)